# Подготовка данных по автоклаву AC101

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from datetime import datetime

Подготовленные исходные данные

In [4]:

ac_data = "merge\автоклавы, сорт. по времени\merge101sortedbydate.xlsx"
chem_in = "merge\хим. анализ\!chem баки питания.xlsx"
chem_out = "merge\хим. анализ\!chem окисленная пульпа.xlsx"
chem_fe2 = "merge\хим. анализ\!chem Fe2+.xlsx"

In [5]:
WORK_DIR = 'working_data'

In [6]:
def file_save(df: pd.DataFrame, folder: str, name: str) -> None:
    comp = {
        'method': 'zip',
        'archive_name': 'out.csv'
    }

    filepath = Path(f'./{folder}/{name}.zip')

    df.to_csv(filepath, compression=comp)

## Подготовка данных

### Данные АСУТП

In [57]:
df_ac = pd.read_excel(ac_data, index_col=1, parse_dates=True)
df_ac.drop("Unnamed: 0", axis=1, inplace=True)

In [58]:
df_ac.isna().sum()

320PU101             17
320PU103             17
320RC101DIC02        17
330AC101FI06         17
330AC101FI07         17
330AC101FI08         17
330AC101FI09         17
330AC101FI10         17
330AC101FI51         17
330AC101FI52         17
330AC101FIC40        17
330AC101FICZ01       17
330AC101FICZ02       17
330AC101FICZ03       17
330AC101FICZ04       17
330AC101FICZ05       17
330AC101LICZ01       17
330AC101LICZ01.1     17
330AC101LICZ01.2     17
330AC101PI64         17
330AC101PI65         17
330AC101PIC63A       17
330AC101PIC63A.1     17
330AC101PIZ61        17
330AC101TIC01        17
330AC101TIC02        17
330AC101TIC03        17
330AC101TIC04        17
330AC101TIC05        17
330AC101TIZ61        17
330PT101PIZ01        17
330TK101LIC01        17
330TK101PI11         17
330TK101TI03         17
330TK101TIC02        17
330TK102LIC01        17
330TK102PI02         17
330TK102TIC02        17
380HX101FIC01        17
380HX101LIC01        17
380HX101PIC03        17
 OXYGEN/OxyPurit

Уберем пропуски

In [59]:
df_ac.dropna(inplace=True)

Усредним все данные с 10 минутных до часовых. Для дальнейшей стыковки с данными ХА.

In [60]:
df_ac_1h = df_ac.resample('H').mean()

In [61]:
df_ac_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26298 entries, 2019-01-01 00:00:00 to 2021-12-31 17:00:00
Freq: H
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   320PU101           24623 non-null  float64
 1   320PU103           24623 non-null  float64
 2   320RC101DIC02      24623 non-null  float64
 3   330AC101FI06       24623 non-null  float64
 4   330AC101FI07       24623 non-null  float64
 5   330AC101FI08       24623 non-null  float64
 6   330AC101FI09       24623 non-null  float64
 7   330AC101FI10       24623 non-null  float64
 8   330AC101FI51       24623 non-null  float64
 9   330AC101FI52       24623 non-null  float64
 10  330AC101FIC40      24623 non-null  float64
 11  330AC101FICZ01     24623 non-null  float64
 12  330AC101FICZ02     24623 non-null  float64
 13  330AC101FICZ03     24623 non-null  float64
 14  330AC101FICZ04     24623 non-null  float64
 15  330AC101FICZ05     24623 no

Некоторые даты пропущены в данных, уберем пропуски еще раз.

In [62]:
df_ac_1h.dropna(inplace=True)

### Данные исходного ХА по бакам питания

In [63]:
df_chem_in = pd.read_excel(chem_in, index_col=0, parse_dates=True, usecols="B:H")
df_chem_in = df_chem_in.apply(pd.to_numeric, errors='coerce')
df_chem_in.fillna(0, inplace=True)

In [64]:
df_chem_in.head()

,320 RC-101 S Fe,320 RC-101 S Sобщ,320 RC-101 S Sso42-,320 RC-101 S As,320 RC-101 S Сорг,320 RC-101 S Собщ
Timestamp,,,,,,
2018-12-01 09:00:00,27.10,25.4,0.0,8.90,0.47,1.34
2018-12-02 09:00:00,25.99,26.9,0.0,7.95,0.50,1.33
2018-12-02 13:00:00,25.78,27.2,0.0,7.95,0.48,1.32
2018-12-02 17:00:00,26.71,26.2,0.0,8.38,0.43,1.22
2018-12-03 20:00:00,0.00,25.9,0.0,8.79,0.64,1.10


In [65]:
df_chem_in_1h = df_chem_in.resample('H', convention='start').first()

In [66]:
df_chem_in_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29075 entries, 2018-12-01 09:00:00 to 2022-03-26 19:00:00
Freq: H
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   320 RC-101 S Fe      1502 non-null   float64
 1   320 RC-101 S Sобщ    1502 non-null   float64
 2   320 RC-101 S Sso42-  1502 non-null   float64
 3   320 RC-101 S As      1502 non-null   float64
 4   320 RC-101 S Сорг    1502 non-null   float64
 5   320 RC-101 S Собщ    1502 non-null   float64
dtypes: float64(6)
memory usage: 1.6 MB


In [67]:
cols = ['Fe', 'Stot', 'SO4', 'As', 'Corg', 'Ctot']
df_chem_in_1h.columns = cols

Приведем даты массивов к одному времени. Возьмем ХА только за те дни, что есть данные с АСУТП

In [68]:
# min_date = df_ac_1h.index.min()
# max_date = df_ac_1h.index.max()

min_date = datetime.fromisoformat('2019-01-01 00:00:00')
max_date = datetime.fromisoformat('2021-12-31 17:00:00')

df_chem_in_1h = df_chem_in_1h[min_date:max_date]

Уберем нулевые значения, а вместо них поставим np.nan

In [69]:
df_chem_in_1h.replace({0: np.NaN}, inplace=True)

### Данные ХА по Fe2+

In [70]:
df_chem_fe2 = pd.read_excel(chem_fe2, index_col=0, parse_dates=True, usecols="B:C")
df_chem_fe2 = df_chem_fe2.apply(pd.to_numeric, errors='coerce')
df_chem_fe2.fillna(0, inplace=True)

In [71]:
df_chem_fe2_1h = df_chem_fe2.resample('H').first()

In [72]:
df_chem_fe2_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29237 entries, 2018-11-29 15:00:00 to 2022-03-31 19:00:00
Freq: H
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   330-ТК-102 L Fe2+  14781 non-null  float64
dtypes: float64(1)
memory usage: 456.8 KB


In [73]:
df_chem_fe2_1h = df_chem_fe2_1h[min_date:max_date]

In [74]:
df_chem_fe2_1h.fillna(method='ffill', inplace=True)

In [75]:
cols = ['Fe2+']
df_chem_fe2_1h.columns = cols

## Данные ОТК по плотностям твердого и питания автоклава

In [10]:
path_19 = r"merge\Nikitos19.xlsx"
path_20 = r"merge\Nikitos20.xlsx"
path_21 = r"merge\Nikitos21.xlsx"

ds_19 = pd.read_excel(path_19, sheet_name='AC101', usecols='B:D', index_col=0, parse_dates=True)
ds_20 = pd.read_excel(path_20, sheet_name='AC101', usecols='B:D', index_col=0, parse_dates=True)
ds_21 = pd.read_excel(path_21, sheet_name='AC101', usecols='B:D', index_col=0, parse_dates=True)

In [11]:
ds_full = pd.concat([ds_19, ds_20, ds_21])

In [12]:
assert(ds_19.shape[0] + ds_20.shape[0] + ds_21.shape[0] == ds_full.shape[0])

In [13]:
ds_full = ds_full.apply(pd.to_numeric, errors='coerce')

In [14]:
ds_full_1h = ds_full.resample('H', convention='start').first()

In [15]:
ds_full_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25573 entries, 2019-01-01 07:00:00 to 2021-12-01 19:00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   D(S)    1682 non-null   float64
 1   D(SL)   1142 non-null   float64
dtypes: float64(2)
memory usage: 599.4 KB


## Финальные проверки массивов данных

In [231]:
# assert(len(df_chem_in_1h) == len(df_ac_1h))
# assert(len(df_chem_fe2_1h) == len(df_ac_1h))

In [82]:
print(f'Начальная дата: {min_date}, конечная дата: {max_date}.')

print(f'Данные автоклава от: {df_ac_1h.index.min()} до {df_ac_1h.index.max()}')
print(f'Данные баков питания от: {df_chem_in_1h.index.min()} до {df_chem_in_1h.index.max()}')
print(f'Данные по железу 2+ от: {df_chem_fe2_1h.index.min()} до {df_chem_fe2_1h.index.max()}')
print(f'Данные по плотности в баках питания от: {ds_full_1h.index.min()} до {ds_full_1h.index.max()}')


print(f'Число точек по автоклаву {len(df_ac_1h)}, по бакам питания {len(df_chem_in_1h)}, по железу {len(df_chem_fe2_1h)} и по плотности {len(ds_full_1h)}')

Начальная дата: 2019-01-01 00:00:00, конечная дата: 2021-12-31 17:00:00.
Данные автоклава от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные баков питания от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по железу 2+ от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по плотности в баках питания от: 2019-01-01 07:00:00 до 2021-12-01 19:00:00
Число точек по автоклаву 24623, по бакам питания 26298, по железу 26298 и по плотности 25573


## Сохранение данных

In [60]:
file_save(df_ac_1h, WORK_DIR, 'df_ac_1h')

In [61]:
file_save(df_chem_in_1h, WORK_DIR, 'df_chem_in_1h')

In [62]:
file_save(df_chem_fe2_1h, WORK_DIR, 'df_chem_fe2_1h')

In [49]:
file_save(ds_full_1h, WORK_DIR, 'ds_full_1h')

Загрузим ранее сохраненные данные, если не изменений ранее

In [21]:
df_ac_1h = pd.read_csv(f'{WORK_DIR}\df_ac_1h.zip', index_col=0, parse_dates=True)
df_chem_in_1h = pd.read_csv(f'{WORK_DIR}\df_chem_in_1h.zip', index_col=0, parse_dates=True)
df_chem_fe2_1h = pd.read_csv(f'{WORK_DIR}\df_chem_fe2_1h.zip', index_col=0, parse_dates=True)

## Объединение данных в одну таблицу

In [22]:
df1 = df_ac_1h.merge(df_chem_in_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(df_chem_fe2_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(ds_full_1h, how='inner', left_index=True, right_index=True)

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23898 entries, 2019-01-01 07:00:00 to 2021-12-01 19:00:00
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   320PU101           23898 non-null  float64
 1   320PU103           23898 non-null  float64
 2   320RC101DIC02      23898 non-null  float64
 3   330AC101FI06       23898 non-null  float64
 4   330AC101FI07       23898 non-null  float64
 5   330AC101FI08       23898 non-null  float64
 6   330AC101FI09       23898 non-null  float64
 7   330AC101FI10       23898 non-null  float64
 8   330AC101FI51       23898 non-null  float64
 9   330AC101FI52       23898 non-null  float64
 10  330AC101FIC40      23898 non-null  float64
 11  330AC101FICZ01     23898 non-null  float64
 12  330AC101FICZ02     23898 non-null  float64
 13  330AC101FICZ03     23898 non-null  float64
 14  330AC101FICZ04     23898 non-null  float64
 15  330AC101FICZ05     23898 non-null  

Сдвинем данные по $Fe2+$ на 2 часа назад.

In [24]:
df1.head()

,320PU101,320PU103,320RC101DIC02,330AC101FI06,330AC101FI07,330AC101FI08,330AC101FI09,330AC101FI10,330AC101FI51,330AC101FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-01 07:00:00,39.395000,0.0,1650.615707,18.998168,20.320517,15.469354,1.912728,0.004103,5308.464948,56.703037,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.23,3900.0,1600.0
2019-01-01 08:00:00,39.400556,0.0,1651.312554,18.982688,20.534195,15.498610,2.105368,0.001700,5308.080298,57.120547,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.45,NaN,NaN
2019-01-01 09:00:00,39.398333,0.0,1653.447314,19.088036,20.343886,15.712674,2.305624,0.025356,5307.429711,57.473187,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.45,NaN,NaN
2019-01-01 10:00:00,39.383333,0.0,1649.847311,18.950787,20.507571,15.410165,1.930532,0.084392,5310.106883,56.887133,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.11,NaN,NaN
2019-01-01 11:00:00,39.406111,0.0,1652.307998,18.859231,20.232325,15.529065,2.205978,0.056138,5313.755440,56.881265,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.11,NaN,NaN


In [25]:
df1['Fe2+'] = df1['Fe2+'].shift(-2)
# df1.dropna(axis=0, inplace=True)

In [26]:
df1.head()

,320PU101,320PU103,320RC101DIC02,330AC101FI06,330AC101FI07,330AC101FI08,330AC101FI09,330AC101FI10,330AC101FI51,330AC101FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-01 07:00:00,39.395000,0.0,1650.615707,18.998168,20.320517,15.469354,1.912728,0.004103,5308.464948,56.703037,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.45,3900.0,1600.0
2019-01-01 08:00:00,39.400556,0.0,1651.312554,18.982688,20.534195,15.498610,2.105368,0.001700,5308.080298,57.120547,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.11,NaN,NaN
2019-01-01 09:00:00,39.398333,0.0,1653.447314,19.088036,20.343886,15.712674,2.305624,0.025356,5307.429711,57.473187,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.11,NaN,NaN
2019-01-01 10:00:00,39.383333,0.0,1649.847311,18.950787,20.507571,15.410165,1.930532,0.084392,5310.106883,56.887133,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.23,NaN,NaN
2019-01-01 11:00:00,39.406111,0.0,1652.307998,18.859231,20.232325,15.529065,2.205978,0.056138,5313.755440,56.881265,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.23,NaN,NaN


In [27]:
file_save(df1, WORK_DIR, 'df_total_ds_1h')

### Проверим данные по исходникам